In [13]:
import sys
import numpy as np
import matplotlib.pyplot as plt
#import uproot
import pandas as pd
import scipy.optimize as opt
import math
import statistics as st
import os
import PyQt5
from numba import njit
import time
import numba
from numba.typed import List
from numba import uint16, intc, uint32
from array import array

# print(os.getcwd())
os.chdir('D:/LANL/')
datadir = 'sample_data/'
folder = 'La_sample/'
run_num = "11139" 
# print(os.getcwd() + folder)
statefileloc = 'D:\LANL\SF_Norm_files\TR_R_expected_avgs_stds_afterclip.csv'
SFNormFile = 'SF_Norm_files/'+folder+run_num
processedpulsefolder = 'processed_data/pulses_added/'
processedasymfolder = 'processed_data/asym/'
AddedPulseSavename = processedpulsefolder+run_num+'_pulsesadded_d'
AsymSavename = processedasymfolder+run_num+'_asym_d'

%matplotlib qt

start = time.time()
fullstart = time.time()

## cannot handle all 24 detectors at once, memory issue... can look into np.empty and deleting variables if needed
#chan_enab = np.array([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24]) ## all
chan_enab = np.array([0,1,2,3,4,5,6,7,8,9,10,11,24]) ## downstream
#chan_enab = np.array([12,13,14,15,16,17,18,19,20,21,22,23,24]) ## upstream
#chan_enab = np.array([0,1,2,3,4,5,18,19,20,21,22,23,24])
#chan_enab = np.array([1,19,24])


#@jit(nopython = True)
# read_data = np.array([])
# fileLength = np.array([])
read_data = []
fileLength = []


def open_file():
    for el in chan_enab:
        f = open(datadir + folder + 'run' + run_num + "_ch" + str(el) + ".bin", 'rb')
        read_data.append(np.fromfile(file=f, dtype=np.uint16))
        f.close()
        fileLength.append(len(read_data[-1]))
    return read_data, fileLength

open_file()

fileLength = np.asarray(fileLength)
read_data = np.asarray(read_data) ## in detector's case, all are the same size samples, so can do read_data as np array

if chan_enab[-1] != 24:
    raise Exception('last channel is not 6Li detector')

end = time.time()
# print('file open time: ' + str(end-start))            

print('processing data: ' + folder + 'run' + run_num)
print("Channel is " + str(chan_enab))
print('saving processed data to ' + AsymSavename)
end = time.time()
print(end-start)
# print(read_data)

processing data: La_sample/run11139
Channel is [ 0  1  2  3  4  5  6  7  8  9 10 11 24]
saving processed data to processed_data/asym/11139_asym_d
1.9716792106628418


Store the big header for each channel in arrays

In [14]:
BoardID = []
recordLength = []
numSamples = []
eventCounter = []
decFactor = []
chanDec = []
postTrig = []
groupStart = []
groupEnd = []
timestamp= []
sizeFirstEvent = []
TTT = []

targetDict = {0: "La", 1: "Tb2O3", 2: "Yb2O3", 3: "Sm2O3", 4: "Er2O3", 5: "Ho2O3", 6: "other"}
foilDict = {0: "TBD", 1: "TBD", 2: "TBD", 3: "TBD", 4: "TBD", 5: "TBD", 6: "other"}

target=(read_data[0][5]&0x00F0)>>4
foil=read_data[0][5]&0x000F
targetFlag = read_data[0][5]>>8&1
foilFlag = read_data[0][5]>>9&1
spinFiltFlag = read_data[0][5]>>10&1
spinFlipFlag = read_data[0][5]>>11&1
shutterFlag = read_data[0][5]>>12&1
facilityTrigFlag = read_data[0][5]>>13&1

if targetFlag:
    target=targetDict[(read_data[0][5]&0x00F0)>>4]
    
else:
    target = "empty"
    
    
if foilFlag:
    foil=foilDict[read_data[0][5]&0x000F]
else:
    foil = "empty"

for i in range(0,len(chan_enab)):
    BoardID.append(read_data[i][9]>>8)
    recordLength.append(((read_data[i][9]&0x00FF)<<16)+read_data[i][8])
    numSamples.append(((read_data[i][11]&0x00FF)<<16)+read_data[i][10])
    eventCounter.append(read_data[i][6]+(read_data[i][7]<<16))
    BoardID.append(read_data[i][9]>>8)  
    decFactor.append(read_data[i][11]>>8)
    chanDec.append(read_data[i][13]>>8)
    postTrig.append(read_data[i][15]>>8)
    groupStart.append(((read_data[i][13]&0x00FF)<<16)+read_data[i][12])
    groupEnd.append(((read_data[i][15]&0x00FF)<<16)+read_data[i][14])
    
    timestamp.append(read_data[i][16]+(read_data[i][17]<<16)+(read_data[i][18]<<32)+(read_data[i][19]<<40))  
    sizeFirstEvent.append(read_data[i][0]+(read_data[i][1]<<16))
    TTT.append(read_data[i][2]+(read_data[i][3]<<16)+(read_data[i][4]<<32))
    
#     print("For channel " + str(chan_enab[i]) + ", BoardID is " + str(BoardID[i])
#           + "; record length is " + str(recordLength[i]) + "; num Samples is " 
#           + str(numSamples[i]) + "; event counter is " + str(eventCounter[i]) + "; dec factor is " + str(decFactor[i]) + "; chan dec is " 
#           + str(chanDec[i]) + "; postTrig is " + str(postTrig[i]) + "; group start is " + str(groupStart[i]) + "; group end is " + str(groupEnd[i])
#           + "; epoch time is " + str(timestamp[i]) +  "; first event size is " + str(sizeFirstEvent[i]) + "; and ETTT is " + str(TTT[i]) + "\n")

BoardID = np.asarray(BoardID) 
recordLength = np.asarray(recordLength)
numSamples = np.asarray(numSamples)
eventCounter = np.asarray(eventCounter)
decFactor = np.asarray(decFactor)
chanDec = np.asarray(chanDec)
postTrig = np.asarray(postTrig)
groupStart = np.asarray(groupStart)
groupEnd = np.asarray(groupEnd)
timestamp = np.asarray(timestamp)
sizeFirstEvent = np.asarray(sizeFirstEvent)
TTT = np.asarray(TTT)

print("Target is " + target)
# print("Foil is " + foil)
# print("Shutter is open: " + str(bool(shutterFlag)))
# print("Facility t0 is on: " + str(bool(facilityTrigFlag)))
# print("Spin flipper is on: " + str(bool(spinFlipFlag)))
# print("Spin filter is on: " + str(bool(spinFiltFlag)))
# print("Target is present: " + str(bool(targetFlag)))
# print("Foil is present: " + str(bool(foilFlag)))


Target is La


Determine the time axis for each channel

In [15]:
preTime = []
startTime = []
endTime = []
resolution = []
xs = [] 

for i in range(0,len(chan_enab)):
    preTime.append((100-postTrig[i])*recordLength[i]/100)
    startTime.append((-1*preTime[i]*16*decFactor[i] + groupStart[i]*16*decFactor[i]))
    endTime.append((-1*preTime[i]*16*decFactor[i] + groupEnd[i]*16*decFactor[i]))
    resolution.append(16*chanDec[i]*decFactor[i])
#     print("Pretime for channel", chan_enab[i],"is " + str(preTime[i]) + "; start time is " + str(startTime[i]) + "; end time is " + str(endTime[i]) 
#           + "; resolution is " + str(resolution[i]) + "ns")
    xs.append(np.arange(startTime[i],(numSamples[i])*resolution[i]+startTime[i], resolution[i]))

#np.asarray(preTime)
#np.asarray(startTime)
#np.asarray(endTime)
#np.asarray(resolution)
xs = np.asarray(xs) ## can convert xs to np array here because all detectors same numsamples


In [16]:
start=time.time()

@njit
def dataread(data, channels, fileLen, numSamps):
    numRuns = int((fileLen[0]-20-numSamps[0])/(numSamps[0]+6)+1)
    ys_arr = np.zeros((len(channels), numRuns,numSamps[0]), dtype=np.uint16)
    ETTT_arr = np.zeros((len(channels), numRuns), dtype=np.intc)
    eventcount_arr = np.zeros((len(channels), numRuns), dtype=np.intc)
    for i in range(0,len(channels)):
        eventCount = 0
        byteCounter = 0
            #byte counter is really 2bytecounter, lol
        while byteCounter < fileLen[i]:
            if byteCounter == 0:
                ETTT_arr[i]=TTT[i]
                #ETTT_arr[i].append(TTT[i])
                eventcount_arr[i]=(eventCounter[i])
                byteCounter = 20
            else:
                ETTT_arr[i]=(data[i][byteCounter]+(data[i][byteCounter+1]<<16)+(data[i][byteCounter+2]<<32))
                eventcount_arr[i]=(data[i][byteCounter+4]+(data[i][byteCounter+5]<<16))
                byteCounter += 6
            for j in range(0, numSamps[i]):
                #if j == 0:
                    #ys_arr[i].append([])
                #print(byteCounter)
                ys_arr[i][eventCount][j]=data[i][byteCounter]
                byteCounter += 1
            eventCount += 1
    return ys_arr, ETTT_arr, eventcount_arr

#start=time.time()
#ys_arrHe, ETTT_arrHe, eventcount_arrHe  = dataread(read_data, [25], fileLength, numSamples) ##hardcoded channel 25 for He
ys_arr, ETTT_arr, eventcount_arr  = dataread(read_data, chan_enab, fileLength, numSamples) ##hardcoded channels for coils

end = time.time()
print('dataread from binary time: ' + str(end-start))            


dataread from binary time: 1.6659696102142334


In [17]:
timeDif=[]
for i in range(0,len(chan_enab)):
    timeDif.append([])
    for j in range(len(ETTT_arr[i])-1):
        timeDif[i].append((ETTT_arr[i][j+1]-ETTT_arr[i][j])*8)
#     print("Min time difference for channel", chan_enab[i], "is", min(timeDif[i]), "ns")
#     print("Max time difference for channel", chan_enab[i], "is", max(timeDif[i]), "ns \n")
#print(timeDif)

In [18]:
## basesub and plotting ##
baseL = 0
baseR = int(((preTime[0]-groupStart[0])*0.70)/chanDec[0])
numRuns = int((fileLength[0]-20-numSamples[0])/(numSamples[0]+6)+1)
    
legend =  ['NaI', 'R']

start = time.time()

s = 20 ## pulse to look at
t=s+1

## dont know why this is so slow ##
def plotter(ys, xs, baseR, numpoints):
    tempys_basesub = np.zeros((len(ys), numRuns,numpoints[0]), dtype=float)
    for i in range((len(ys))):
        for pulse in range((len(eventcount_arr[0]))): ## all have 5000 pulses
            tempys_basesub[i][pulse]=np.subtract(ys[i][pulse], np.mean(ys[i][pulse][baseL:baseR]))
        for j in range(s, t): ## plot only interested pulses
            plt.plot(xs[i], tempys_basesub[i][j]) #label=legend[i]) #+str(sums[1][j])) ## sums[j] will not work for more than just TR   
            plt.axvline(xs[0][baseL], ls = '--')
            plt.axvline(xs[0][baseR], ls = '--')
            #plt.axvline(xs[0][int(((preTime[0]-groupStart[0])*0.70)/chanDec[0])], ls = '--', c ='m')
            plt.axvline(xs[0][baseR+5], ls = '--', c ='r') ## BaseR+5 line marks the beginning of the integral, until the end of samples.
#             plt.title('SF state transition' + transitions[p]) 
#             plt.xlabel("time from trigger (ns)")
#             plt.ylabel("ADC")
            plt.legend()
            
#plotter(ys_arr[9:], xs[9:], baseR, numSamples) ##plot coils

ys_basesub = np.zeros((len(ys_arr), numRuns,numSamples[0]), dtype=np.float64)

@njit ## jit is faster for large # channels, slower for small # channels
def basesub(ys, baseRight, numpoints): 
    tempys_basesub = np.zeros((numRuns,numpoints[0]), dtype=np.float64)
    for pulse in range((len(eventcount_arr[0]))): ## all have 5000 pulses
        tempys_basesub[pulse]=np.subtract(ys[pulse], np.mean(ys[pulse][baseL:baseRight]))
    return tempys_basesub

## got rid of sums here, should be done after aligning and cutting
## got rid of xs in basesub, don't think we need them as an input 06.10.24

for i in range(len(ys_basesub)):
    ys_basesub[i] = basesub(ys_arr[i], baseR, numSamples)

ys_basesub[-1] = ys_basesub[-1]*-1 ## invert 6Li to positive signal. Comment out if not using

end = time.time()
print('plotting and/or base subtraction time: ' + str(end-start))            

plotting and/or base subtraction time: 4.539802551269531


## Load SF Sorting and norm ##

In [19]:
## Load in SF and He normalization information 
df_SF = pd.read_hdf(SFNormFile + '.h5', key='df_0')
df_HE = pd.read_hdf(SFNormFile + '.h5', key='df_1')

SF_Sort_arr = df_SF[['nicknames', 'transition_locations']].to_numpy().T
He_Norm_arr = df_HE[['pulse', 'norms']].to_numpy().T

NormFactor = 100000  ## He integrals are huge, this normalizes all of those by a constant value for ease of use
HeNorms= (He_Norm_arr[1])/NormFactor


In [20]:
# for ind in df.index:
#     print('transition: '+ str(df['nicknames'][ind]) + ' location: ' + str(df['transition_locations'][ind]))
#print(df['transition_locations'],df['nicknames'] )

counter = 0
sequence = [[],[],[]]
seq = 0
smallerseq = []
smallerstateis = []

for i in range(len(SF_Sort_arr[1])-(np.mod((len(SF_Sort_arr[1])), 8))):  ##111 mod 8 = 7, so essentially 111-7 = 104
    counter = counter+1
    if counter < 8:
        smallerstateis.append([(SF_Sort_arr[1][i])+5,(SF_Sort_arr[1][i+1])])
        smallerseq.append(SF_Sort_arr[0][i+1])
    elif counter == 8:
        seq = seq+1
        smallerstateis.append([(SF_Sort_arr[1][i])+5,(SF_Sort_arr[1][i+1])])
        smallerseq.append(SF_Sort_arr[0][i+1])
        sequence[0].append(seq)
        sequence[1].append(smallerseq)   
        sequence[2].append(smallerstateis)
        smallerseq = []
        smallerstateis = []
        counter  = 0

leftovers = [[sequence[0][-1]+1],[],[]]

for i in range((len(SF_Sort_arr[1])-(np.mod((len(SF_Sort_arr[1])), 8))), len(SF_Sort_arr[1])-1):
    counter = counter+1
    if counter < 8:
        leftovers[1].append(SF_Sort_arr[0][i+1])
        leftovers[2].append([(SF_Sort_arr[1][i])+5,(SF_Sort_arr[1][i+1])])
    elif counter == 8: ## not really needed here, maybe delete
        seq = seq+1
        leftoversum.append(np.sum(sums_normed[0][(SF_Sort_arr[1][i])+5:(SF_Sort_arr[1][i+1])]))
        smallerseq.append(SF_Sort_arr[0][i+1])
        sequence[0].append(seq)
        sequence[1].append(smallerseq)       
        smallerseq = []
        counter  = 0

print(sequence[0])
print(sequence[1][0])
print(leftovers)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
[5, 6, 7, 0, 1, 2, 3, 4]
[[14], [5, 6, 7, 0, 1, 2], [[4720, 4760], [4765, 4805], [4810, 4850], [4855, 4895], [4900, 4940], [4945, 4985]]]


## t0 aligning ##

In [21]:
## use 6Li t0 for all instead of for themselves individually

start = time.time()

NaIthresh=2000
Li6thresh=1000

threshold_array = (np.full(len(ys_basesub), NaIthresh))
threshold_array[-1] = Li6thresh

#@njit ## numba does not support reversed, but this could be changed if it's slow
def find_offset(ys, thresharr):
    xCrosses = np.zeros((len(ys), numRuns)) #outer array is crossing arrays for given channel, inner array is crossing for each event
    offset = np.zeros((len(ys), numRuns), dtype=np.int32) ##offset in bins for each channel, each pulse
    modeCrosses = np.zeros((len(ys)), dtype=np.float64)
    for i in reversed(range(len(ys))):
        #xValues.append([])
        for p in range(len(ys[i])):
            xing = np.argmax(ys[i][p] > thresharr[i])
            #print(xing)
            xCrosses[i][p] = xing
        modeCrosses[i] = (st.mode(xCrosses[i])) #find the most typical crossing value for each channel
        for p in range(len(xCrosses[i])):
            offset[i][p] = (modeCrosses[-1] - xCrosses[i][p]) ## make sure this is the correct sign!!! 
    if (np.all(xCrosses[-1])) == False:
        raise Exception('ERROR: 6Li threshold was not reached for at least one pulse')
    return offset, xCrosses, modeCrosses
                           
offset, xCrosses, modeCrosses = find_offset(ys_basesub, threshold_array)

end = time.time()
print('finding offset time: ' + str(end-start))            

finding offset time: 1.1737792491912842


In [10]:
## this cell loops through every channel as opposed to inputting all channels at once. 5x faster

start = time.time()

## extend all arrays by a value, check that the max number of offset on 6Li is less than that value ##
extendedRange = 3 ## must be a positive value which to extend ys_arr
if abs(max(offset[-1], key = abs)) > extendedRange: ## if the max offset of 6Li is >extendedRange, something is wrong
    raise Exception('ERROR: largest offset greater than extended range')

ys_ext = np.zeros((len(ys_basesub), len(ys_basesub[0]), len(ys_basesub[0][0])+extendedRange*2), dtype=np.float64)
ys_cut = np.zeros((len(ys_basesub), len(ys_basesub[0]), (len(ys_ext[0][0])-((extendedRange*2)+1)*2)))
xs_cut = np.zeros((len(ys_cut), len(ys_cut[0][0])))

# cant use jit because np.pad is not supported
def align_cut_norm(ys, xs_arr, extendedr):
    tempys_ext = np.zeros((len(ys), len(ys[0])+extendedr*2), dtype=np.float64)
    tempys_cut = np.zeros((len(ys), (len(tempys_ext[0])-((extendedr*2)+1)*2)))
    tempxs_cut = np.zeros(len(tempys_cut[0]))
    for p in range(len(ys)):
        tempys_ext[p] = np.pad(ys[p], extendedr, 'constant', constant_values=(0))
        tempys_ext[p] = np.roll(tempys_ext[p],offset[-1][p]) ## assumes 6Li at -1 position
        tempys_cut[p] = tempys_ext[p][((extendedr*2)+1):-((extendedr*2)+1)].copy() ## cut by 7 (if extRange == 3)
        tempys_cut[p] = tempys_cut[p]/HeNorms[p] ## normalize by 3He integral
    x_cut_amt = int((len(ys[0]) - len(tempys_cut[0]))/2)
    tempxs_cut = xs_arr[x_cut_amt:-x_cut_amt].copy()
    return tempys_cut, tempxs_cut

for i in range(len(ys_basesub)):
    ys_cut[i], xs_cut[i] = align_cut_norm(ys_basesub[i], xs[i], extendedRange)
    
# checkp = 2053
# print(offset[-1][checkp]) ## checking offset for one example checkpulse
# print('original index for checkpulse: '+str(np.argmax(ys_basesub[0][checkp]> 2000))) ## we can follow the index as it changes with extension/cut
# #print('extended range index for checkpulse: '+str(np.argmax(ys_ext[0][checkp]> 2000)))
# print('cut array index for checkpulse: '+str(np.argmax((ys_cut[0][checkp]*HeNorms[checkp])> 2000)))

del ys_ext ## might help with memory issues
del ys_basesub

end = time.time()
print('aligning and cutting time: ' + str(end-start))            

aligning and cutting time: 8.645994901657104


## add up N number of pulses

In [11]:
## add up pulses for their respective state, in each 8 step sequence
## turning into a by-channel function 06.13.24

start = time.time()

legend = ['NaI5 (downstream)','NaI4R (upstream)','6Li']

## t_to_eV = 0.5*(1.6749286*10**(-27))*((27.3/TIME))**2*(6.24151*10**18 [J/eV])

added_pulses = np.zeros((len(ys_cut), len(sequence[0]), 8, len(ys_cut[0][0])), dtype=np.float64) ## 13 sequences, 8 stages each works?
## i channels, 13 sequences each, 8 states each sequence, 8992 num points

ON_OFF_sums = np.zeros((len(ys_cut), len(sequence[0]), 2, len(ys_cut[0][0])), dtype=np.float64) ## 13 sequences, 2 for ON or OFF for each sequence

#@njit
def add_pulse(ys, SFarr):
    tempadded_p = np.zeros((len(SFarr[0]), 8, len(ys[0])), dtype=np.float64)    
    temp_ONOFF = np.zeros((len(SFarr[0]), 2, len(ys[0])), dtype=np.float64)
    for seq in range(0, len(SFarr[0])): ## for every sequence
        for state in range(0, len(SFarr[1][0])): ## for every state in the sequence
            for p in range((SFarr[2][seq][state][0]),(SFarr[2][seq][state][1])+1): ##not sure if this works. from 20-60 for example
#                 tempadded_pulses[i] = added_pulses[i]+np.array(ys_cut[i][j])
                tempadded_p[seq][state] = np.add(tempadded_p[seq][state],(ys[p])) ## start with zeros, add to each iteratively
            if state==0 or state==3 or state==5 or state==6: ## these are ON states
                temp_ONOFF[seq][0] = np.add(temp_ONOFF[seq][0],(tempadded_p[seq][state])) ## start with zeros, add to each iteratively
            if state==1 or state==2 or state==4 or state==7: ## these are OFF states
                temp_ONOFF[seq][1] = np.add(temp_ONOFF[seq][1],(tempadded_p[seq][state])) ## start with zeros, add to each iteratively
    return tempadded_p, temp_ONOFF
                
for i in range(len(ys_cut)):
    added_pulses[i], ON_OFF_sums[i] = add_pulse(ys_cut[i], sequence)
                
## plotting examples
# plt.plot(xs_cut[i], added_pulses[0][0][0] , label=legend[0] +', sequence 1 state 1, 40 pulses added')
# plt.plot(xs_cut[i], added_pulses[0][0][1] , label=legend[0] +', sequence 1 state 2, 40 pulses added')
# plt.plot(xs_cut[i], added_pulses[0][1][0] , label=legend[0] +', sequence 2 state 1, 40 pulses added') 
# plt.plot(xs_cut[i], added_pulses[0][1][1] , label=legend[0] +', sequence 2 state 2, 40 pulses added') 
    
# plt.title('Detector signals') 
# plt.xlabel("time from trigger (ns)")
# plt.ylabel("ADC")

# # plt.axvline(xs[0][baseL], ls = '--')
# # plt.axvline(xs[0][baseR], ls = '--')
# #plt.axvline(xs[1][intgrL], ls = '--', c ='g')
# #plt.axvline(xs[1][intgrR], ls = '--', c ='g')
# #plt.axvline(xs[2][HeintgrL], ls = '--', c ='r')
# #plt.axvline(xs[2][HeintgrR], ls = '--', c ='r')

# plt.legend()
# plt.show()

print(len(added_pulses[1]))

end = time.time()
print('summing pulses into their states time: ' + str(end-start))  

13
summing pulses into their states time: 0.48624300956726074


In [12]:
start = time.time()

Asym = np.zeros((len(ON_OFF_sums), len(ON_OFF_sums[0][0][0])), dtype=np.float64) ## 1 Asym for each channel, not for each sequence (can change)

def asym(ON_OFF_arr):
    tempasym = np.zeros((len(ON_OFF_arr[0][0])), dtype=np.float64)
    for seq in range(len(ON_OFF_arr[0])): ## number of sequences
        asymform = ((ON_OFF_arr[seq][0]-ON_OFF_arr[seq][1]) / (ON_OFF_arr[seq][0]+ON_OFF_arr[seq][1]))
        tempasym = np.add(asymform,tempasym)
    normedasym = tempasym/len(ON_OFF_sums[0])
    return normedasym

for i in range(len(ON_OFF_sums)):
    Asym[i] = asym(ON_OFF_sums[i])

end = time.time()
print('calculate asymmetry time: ' + str(end-start)) 

calculate asymmetry time: 0.0010063648223876953


In [13]:
np.save(AddedPulseSavename, added_pulses)
np.save(AsymSavename, Asym)

fullend = time.time()
print('full processing time: ' + str(fullend-fullstart))  

full processing time: 19.09673523902893


## end of data processing ##

In [14]:
# ## plotting examples
# # Asym = (ON_OFF_sums[-1][0][0]-ON_OFF_sums[-1][0][1]) / (ON_OFF_sums[0][0][0]+ON_OFF_sums[0][0][1])
# # plt.plot(xs_cut[i], ON_OFF_sums[0][0][0] , label=legend[0] +', sequence 1 ON pulses')
# # plt.plot(xs_cut[i], ON_OFF_sums[0][0][1] , label=legend[0] +', sequence 1 OFF pulses')
# # plt.plot(xs_cut[i], ON_OFF_sums[0][1][0] , label=legend[0] +', sequence 2 ON pulses') 
# # plt.plot(xs_cut[i], ON_OFF_sums[0][1][1] , label=legend[0] +', sequence 2 OFF pulses') 
# plt.plot(xs_cut[2], Asym[2] , label=legend[0] +'NaI asym') 

    
# #plt.xlim(-50000, 900000)
# plt.ylim(-1.5,1.5)
# plt.title('Detector signals') 
# plt.xlabel("time from trigger (ns)")
# plt.ylabel("ADC")

# # plt.axvline(xs[0][baseL], ls = '--')
# # plt.axvline(xs[0][baseR], ls = '--')
# #plt.axvline(xs[1][intgrL], ls = '--', c ='g')
# #plt.axvline(xs[1][intgrR], ls = '--', c ='g')
# #plt.axvline(xs[2][HeintgrL], ls = '--', c ='r')
# #plt.axvline(xs[2][HeintgrR], ls = '--', c ='r')

# #plt.text(1.5e7, 1200, 'test', fontsize=12)
# plt.legend()
# plt.show()